# Flaconi Data Engineering Challenge

## Objectives

This challenge involves mainly the following aspects:
- multidimensional modeling for Data Warehouse
- ETL by using Python
- ability to logically structure your code

## Your task
Create a Python script (*or insert your code directly into this Jupyter notebook if you prefer*) that transforms this Excel file into tables with a structure appropriate for DWH. We suggest the following tables, but feel free to create Fact and Dimension Tables as you see fit:
- fact_sales
- dim_product
- dim_customer

**The script should do the following** (feel free to add additional transformation if you feel they are needed):
- split the dataset into several tables in a Datawarehouse-friendly design (*Note: there is no right or wrong - just explain why you made a specific choice. The thought process and the way of arriving at a solution is more relevant than the answer itself*) 
- get rid of spaces in column names + make column names uppercase without manually adjusting the column names (i.e. do it programatically, don't manually rename columns)
- transform the dataset in such a way that we could use those tables as Fact and Dimension tables in a data warehouse
- **calculate the following KPIs:**
    1. Sum of revenues per month (for the year 2017 - this dataset only includes the year 2017)
    2. The total number of orders per month (the total count)
    3. Average order volume per month
    4. Sum of revenues per customer segment
    5. Top 10 customers, incl. their Lifetime Order Volume (i.e. the lifetime revenue the company made with those customers)

## Dataset
Source: https://data.world/jerrys/sql-project/workspace/data-dictionary, License: Public Domain - open to the world

In [1]:
import pandas as pd
from Scripts.calculate_kpi import get_revenues_sum 
from Scripts.calculate_kpi import get_top_n_customers
from Scripts.calculate_kpi import orders_per_month
from Scripts.ETL import ETL
from Scripts.utils import remove_spaces_and_uppercase_df_columns

pd.set_option("max_columns", 50)
pd.set_option("display.max_colwidth", 150)

sales = pd.read_excel("sales.xlsx", sheet_name='Sales')
sales.head()

,Product ID,Order ID,Order Date,Product name,Order Priority,Order Quantity,Unit Price,Discount,Shipping Price,Total,Total After Discount,Box Size,Shipping Cost,Box Cost,Delivery Date,Product Category,Product Base Margin,Customer Id,Customer Name,Customer Region,Customer Segment,Revenue
0,PFS127009,200920,2017-09-10,"7"" Dog Frisbee",Low,7,40.95,0.04,1.75,286.65,275.18,Jumbo Drum,1.23,5.42,2017-09-18,Pet,0.4206,100922001,Muhammed MacIntyre,Nunavut,Small Business,109.10
1,PFS127010,200921,2017-08-16,Google Infant Short Sleeve Tee Green,Medium,1,23.70,0.07,3.00,23.70,22.04,Small Box,2.10,0.56,2017-08-22,Infant,0.3804,100922002,Barry French,Nunavut,Consumer,5.72
2,PFS127011,200922,2017-11-03,BLM Sweatshirt,High,3,24.30,0.01,6.00,72.90,72.17,Small Box,4.20,0.56,2017-11-06,Apparel,0.3722,100922003,Clay Rozendal,Nunavut,Corporate,22.10
3,PFS127025,200923,2017-04-04,Google Baby Essentials Set,Critical,9,15.60,0.08,7.50,140.40,129.17,Medium Box,5.25,1.18,2017-04-06,Infant,0.6966,100922004,Carlos Soltero,Nunavut,Home Office,83.55
4,PFS127026,200924,2017-03-15,Google 17oz Stainless Steel Sport Bottle,Very Low,12,15.77,0.08,2.33,189.23,174.09,Small Pack,1.63,0.38,2017-03-25,Water Bottles,0.6850,100922005,Carl Jackson,Nunavut,Consumer,117.24


In [2]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18782 entries, 0 to 18781
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Product ID            18782 non-null  object        
 1   Order ID              18782 non-null  int64         
 2   Order Date            18782 non-null  datetime64[ns]
 3   Product name          18782 non-null  object        
 4   Order Priority        18782 non-null  object        
 5   Order Quantity        18782 non-null  int64         
 6   Unit Price            18782 non-null  float64       
 7   Discount              18782 non-null  float64       
 8   Shipping Price        18782 non-null  float64       
 9   Total                 18782 non-null  float64       
 10  Total After Discount  18782 non-null  float64       
 11  Box Size              18782 non-null  object        
 12  Shipping Cost         18782 non-null  float64       
 13  Box Cost        

## Get rid of spaces in column names + make column names uppercase without manually  adjusting the column names (i.e. do it programatically, don't manually rename columns)

In [3]:
remove_spaces_and_uppercase_df_columns(sales)
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18782 entries, 0 to 18781
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   PRODUCTID           18782 non-null  object        
 1   ORDERID             18782 non-null  int64         
 2   ORDERDATE           18782 non-null  datetime64[ns]
 3   PRODUCTNAME         18782 non-null  object        
 4   ORDERPRIORITY       18782 non-null  object        
 5   ORDERQUANTITY       18782 non-null  int64         
 6   UNITPRICE           18782 non-null  float64       
 7   DISCOUNT            18782 non-null  float64       
 8   SHIPPINGPRICE       18782 non-null  float64       
 9   TOTAL               18782 non-null  float64       
 10  TOTALAFTERDISCOUNT  18782 non-null  float64       
 11  BOXSIZE             18782 non-null  object        
 12  SHIPPINGCOST        18782 non-null  float64       
 13  BOXCOST             18782 non-null  float64   

## Transform the dataset in such a way that we could use those tables as Fact and Dimension tables in a data warehouse

In [4]:
dimension_features_without_dimension_name_substring = {'PRODUCT': ['UNITPRICE'],
                                                       'CUSTOMER': [],
                                                       'ORDER':[]}

fact_table_columns_containing_dimension_name = ['ORDERPRIORITY', 'ORDERQUANTITY']
            
e = ETL('starschema')
e.init_params(dataframe_xlsx_path = 'sales.xlsx',
              xlsx_sheet_name = 'Sales',
              dimension_features_without_dimension_name_substring = dimension_features_without_dimension_name_substring,
              fact_table_columns_containing_dimension_name = fact_table_columns_containing_dimension_name)

StarSchema : parameter Initialized!!


In [5]:
e.transform_table()

DIM_PRODUCT Table:  ['UNITPRICE', 'PRODUCTID', 'PRODUCTNAME', 'PRODUCTCATEGORY', 'PRODUCTBASEMARGIN']
Table saved:output\SALES\StarSchema\dim_PRODUCT.parquet
DIM_CUSTOMER Table:  ['CUSTOMERID', 'CUSTOMERNAME', 'CUSTOMERREGION', 'CUSTOMERSEGMENT']
Table saved:output\SALES\StarSchema\dim_CUSTOMER.parquet
DIM_ORDER Table:  ['ORDERID', 'ORDERDATE']
Table saved:output\SALES\StarSchema\dim_ORDER.parquet

Fact Table: ['PRODUCTID' 'ORDERID' 'ORDERPRIORITY' 'ORDERQUANTITY' 'DISCOUNT'
 'SHIPPINGPRICE' 'TOTAL' 'TOTALAFTERDISCOUNT' 'BOXSIZE' 'SHIPPINGCOST'
 'BOXCOST' 'DELIVERYDATE' 'CUSTOMERID' 'REVENUE']
Table saved:output\SALES\StarSchema\fact_SALES.parquet


In [6]:
merged_df = e.get_merged_table()

Location: output\SALES\StarSchema\dim_CUSTOMER.parquet
File Name: dim_CUSTOMER.parquet
Location: output\SALES\StarSchema\dim_ORDER.parquet
File Name: dim_ORDER.parquet
Location: output\SALES\StarSchema\dim_PRODUCT.parquet
File Name: dim_PRODUCT.parquet
Location: output\SALES\StarSchema\fact_SALES.parquet
File Name: fact_SALES.parquet


In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18782 entries, 0 to 18781
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   PRODUCTID           18782 non-null  object        
 1   ORDERID             18782 non-null  int64         
 2   ORDERPRIORITY       18782 non-null  object        
 3   ORDERQUANTITY       18782 non-null  int64         
 4   DISCOUNT            18782 non-null  float64       
 5   SHIPPINGPRICE       18782 non-null  float64       
 6   TOTAL               18782 non-null  float64       
 7   TOTALAFTERDISCOUNT  18782 non-null  float64       
 8   BOXSIZE             18782 non-null  object        
 9   SHIPPINGCOST        18782 non-null  float64       
 10  BOXCOST             18782 non-null  float64       
 11  DELIVERYDATE        18782 non-null  datetime64[ns]
 12  CUSTOMERID          18782 non-null  int64         
 13  REVENUE             18782 non-null  float64   

### 1. Sum of revenues per month (for the year 2017 - this dataset only includes the year 2017)

In [9]:
get_revenues_sum(merged_df, variable = 'month')

,MONTH,REVENUE
0,Jan,353288.61
1,Feb,300610.43
2,Mar,344940.20
3,Apr,358513.21
4,May,403075.57
5,Jun,315658.33
6,Jul,358101.83
7,Aug,365501.18
8,Sep,365658.94
9,Oct,357781.78


### 2. The total number of orders per month (the total count)

In [10]:
orders_per_month(merged_df, aggregate_function='sum')

,MONTH,ORDERSPERMONTH
0,Jan,1563
1,Feb,1439
2,Mar,1597
3,Apr,1559
4,May,1607
5,Jun,1559
6,Jul,1579
7,Aug,1586
8,Sep,1523
9,Oct,1608


### 3. Average order volume per month

In [11]:
orders_per_month(merged_df, aggregate_function='average order volume')

,MONTH,AVERAGEGORDERVOLUMEPERMONTH
0,Jan,1563.000000
1,Feb,1501.000000
2,Mar,1533.000000
3,Apr,1539.500000
4,May,1553.000000
5,Jun,1554.000000
6,Jul,1557.571429
7,Aug,1561.125000
8,Sep,1556.888889
9,Oct,1562.000000


### 4. Sum of revenues per customer segment

In [12]:
get_revenues_sum(merged_df, variable = 'customer segment')

,CUSTOMERSEGMENT,REVENUE
0,Consumer,2883893.77
1,Small Business,553162.09
2,Home Office,538636.57
3,Corporate,329763.97


### 5. Top 10 customers, incl. their Lifetime Order Volume (i.e. the lifetime revenue the company made with those customers)

In [13]:
get_top_n_customers(merged_df, sort_column = 'lifetimeordervolume')

,CUSTOMERID,CUSTOMERNAME,LIFETIMEORDERVOLUME
0,100922787,Alan Dominguez,26
1,100922788,Stephanie Phelps,26
2,100922789,Tonja Turnell,26
3,100922790,Christopher Conant,26
4,100922791,Cyma Kinney,26
5,100922792,Shui Tom,26
6,100922793,Victoria Brennan,26
7,100922794,Adrian Shami,26
8,100922795,Harry Greene,26
9,100922033,Skye Norling,24


In [14]:
get_top_n_customers(merged_df, sort_column = 'LIFETIMEREVENUE')

,CUSTOMERID,CUSTOMERNAME,LIFETIMEREVENUE
0,100922376,Nicole Brennan,12789.24
1,100922723,Jill Matthias,12273.02
2,100922321,Scot Coram,12126.79
3,100922132,Kristina Nunn,11787.36
4,100922322,Natalie Fritzler,11595.34
5,100922384,Evan Henry,11348.34
6,100922114,Stewart Carmichael,11129.72
7,100922375,Paul Knutson,11001.23
8,100922652,Karen Bern,10996.70
9,100922295,George Bell,10676.78
